In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import time

In [2]:
URL = "https://www.rappi.cl/restaurantes"

# Get categories

In [3]:
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

categories = []


job_elements = soup.find_all("span", attrs={"data-testid":"typography", "class":"sc-bxivhb eeSaHG sc-5d042f5c-5 gzdxii"})
for job_element in job_elements:
    elements = job_element.text
    categories.append(elements)
    
categories

['Pizzas',
 'Hamburguesas',
 'Experiencias Culinarias',
 'Sushi',
 'Peruana',
 'Asiática',
 'Pastelería y Postres',
 'Pollo',
 'Mexicana',
 'Árabe',
 'Saludable',
 'Italiana',
 'Helados',
 'Desayunos',
 'Sándwiches',
 'Hot Dogs',
 'Carnes',
 'Vegetariana',
 'Pescados y Mariscos',
 'Comida Casera',
 'Café',
 'Internacional',
 'Jugos y Batidos',
 'Americana']

# Using Categories

In [5]:
driver = webdriver.Chrome() 
driver.implicitly_wait(0.5)
driver.get(URL)
restorants_urls = []


# 2 == Hamb.

driver.find_element(By.XPATH,'//*[@id="__next"]/div[3]/div[3]/div[1]/div/div/button[2]').click()

time.sleep(5)

html = driver.page_source

soup = BeautifulSoup(html, "html.parser")

job_elements = soup.find_all("a", attrs={"data-testid":"store-item"})

for job_element in job_elements:
    restorants_urls.append(job_element["href"])   
    
driver.close()
restorants_urls

['/restaurantes/900031483-chef-metalero',
 '/restaurantes/900038379-juicy-lucy',
 '/restaurantes/900028590-roof-burger',
 '/restaurantes/900067604-venice',
 '/restaurantes/900030543-biteme-organic-burger',
 '/restaurantes/900083275-rayo-burger',
 '/restaurantes/900015883-mr-jack',
 '/restaurantes/900021235-mcdonalds',
 '/restaurantes/900028711-burgery',
 '/restaurantes/900014611-carls-jr',
 '/restaurantes/900015570-burger-kingr',
 '/restaurantes/900073000-fiki-burger',
 '/restaurantes/900014562-doble-queso',
 '/restaurantes/900017651-la-burguesita-by-la-burguesia',
 '/restaurantes/900064834-burger-beef',
 '/restaurantes/900016264-wendys',
 '/restaurantes/900057203-johnny-rockets',
 '/restaurantes/900071197-just-burger',
 '/restaurantes/900031546-bayside-burgers-n-shakes',
 '/restaurantes/900031792-sorry-burger',
 '/restaurantes/900017512-uncle-fletch',
 '/restaurantes/900026555-sonder-meat',
 '/restaurantes/900028490-le-nam',
 '/restaurantes/900029077-la-burgueseria',
 '/restaurantes/9

# Get Data
- Link
- Name
- Address
- Stars
- Total products


In [53]:
results = pd.DataFrame()

counter = 0

for url in restorants_urls:
    new_url = "https://www.rappi.cl{}".format(url)
    page = requests.get(new_url)
    soup = BeautifulSoup(page.content, "html.parser")
    
    print("parsing:", new_url)
    
    name = soup.find("h1", {"class":"chakra-text css-6oec1p"}).text
    address = soup.find("h2", {"class":"sc-bxivhb dVvqfA"}).text
    stars = soup.find("span", {"class":"sc-bxivhb fyzoZN"}).text
        
    try:
        stars = float(stars)
    except ValueError:
        stars = None
    
    
    all_products = soup.find_all("div", {"class":"sc-5a7def68-0 hbVrBp sc-a04fe063-1 hQgbBh"})
    
    obs = {"link": new_url, 
           "name": name, 
           "address": address, 
           "stars": stars,
           "total_products": len(all_products)}
    results = results.append(obs, ignore_index=True)
    
    time.sleep(3)
    counter +=1
    
    if counter > 3:
        break
    
    
        
results

parsing: https://www.rappi.cl/restaurantes/900031483-chef-metalero
parsing: https://www.rappi.cl/restaurantes/900083275-rayo-burger
parsing: https://www.rappi.cl/restaurantes/900038379-juicy-lucy
parsing: https://www.rappi.cl/restaurantes/900030543-biteme-organic-burger


,link,name,address,stars,total_products
0,https://www.rappi.cl/restaurantes/900031483-ch...,Chef Metalero,"Rancagua 574, Providencia",4.3,36
1,https://www.rappi.cl/restaurantes/900083275-ra...,Rayo Burger,"Alonso de Córdova 5320, Las Condes, Chile",NaN,24
2,https://www.rappi.cl/restaurantes/900038379-ju...,Juicy Lucy,"Orrego Luco 145, Providencia, Región Metropoli...",3.4,76
3,https://www.rappi.cl/restaurantes/900030543-bi...,Biteme Organic Burger,avenida Apoquindo 4900 local 112 y 113,NaN,45
